# Chatta med Lär dig Python från grunden

**OBS! Det här är fördjupningsmaterial för den som är intresserad och hör inte till kursens obligatoriska innehåll!**

Här är exempelkod för hur jag byggde en chatbot som svarar på frågor utifrån boken *Lär dig Python från grunden*.

Notera att notebooken inte kommer att gå att köra hur som helst - jag har inte delat min privata API-nyckel, och inte heller boken i PDF-format.

Om du vill testa på eget material kan du skapa en egen API-nyckel och läsa mer [här](https://ai.google.dev/gemini-api/docs).

Koden är alltså enbart till för att demonstrera konceptet med RAG (*Retreival-Augumented Generation*).

Du kan läsa mer om RAG [här](https://aws.amazon.com/what-is/retrieval-augmented-generation/).

In [1]:
import os
import uuid
import google.generativeai as genai
import pandas as pd
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import chromadb

genai.configure(api_key=os.environ.get('API_KEY'))

In [2]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection('ldpfg')

In [3]:
reader = PdfReader('Lär-dig-Python-från-grunden.pdf')

In [4]:
text = ''
for page in reader.pages[19:]:
    text += page.extract_text() 

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

uuids = []
docs = text_splitter.create_documents([text])
for i, d in enumerate(docs):
    uuids.append(str(uuid.uuid4()))
    d.metadata = {"doc_id": i}

In [15]:
import chromadb.utils.embedding_functions as embedding_functions

gem_ef = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ.get('API_KEY'))

In [16]:
doc_texts = [d.page_content for d in docs]

In [17]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.PersistentClient(path='ldpfg.db')
  db = chroma_client.get_or_create_collection(name=name, embedding_function=gem_ef)

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [ ]:
db = create_chroma_db(doc_texts, 'ldpfg_db')

In [19]:
model = genai.GenerativeModel("gemini-1.5-flash")

def build_prompt(query: str, context: list[str]) -> str:
    """
    Builds a prompt for the LLM. #

    This function builds a prompt for the LLM. It takes the original query,
    and the returned context, and asks the model to answer the question based only
    on what's in the context, not what's in its weights.

    Args:
    query (str): The original query.
    context (List[str]): The context of the query, returned by embedding search.

    Returns:
    A prompt for the LLM (str).
    """

    base_prompt = {
        "content": "Jag kommer ställa dig en fråga, och jag vill att du svarar"
        " baserad bara på kontexten jag skickar med, och ingen annan information."
        " Om det inte finns nog med information i kontexten för att svara på frågan,"
        ' säg "Det vet jag inte", och försök att gissa.'
        " Dela upp svaret i fina stycken.",
    }
    user_prompt = {
        "content": f" Frågan är '{query}'. Här är kontexten du får:"
        f'{(" ").join(context)}',
    }

    # combine the prompts to output a single prompt string
    system = f"{base_prompt['content']} {user_prompt['content']}"

    return system


def get_gemini_response(query: str, context: list[str]) -> str:
    """
    Queries the Gemini API to get a response to the question.

    Args:
    query (str): The original query.
    context (List[str]): The context of the query, returned by embedding search.

    Returns:
    A response to the question.
    """

    response = model.generate_content(build_prompt(query, context))

    return response.text


In [26]:
query = "Vad är en exception i Python?"

results = db.query(query_texts=[query], n_results=5)

print(get_gemini_response(query, results["documents"][0]))

Baserat på den givna texten är en exception i Python ett fel som uppstår under körningen av ett program.  Det kan vara av olika typer, såsom `Exception`, `TypeError`, eller `ZeroDivisionError`.

Ett exempel på en exception är när man försöker dela med noll (`5/0`), vilket resulterar i ett `ZeroDivisionError`.  En annan är när man försöker utföra en operation på fel datatyp, exempelvis att addera en sträng till ett heltal (`5 + "Python is fun!"`), vilket leder till en `TypeError`.

Programmeringsspråket Python tillåter att dessa exceptions fångas med hjälp av `try...except`-block.  Detta gör att programmet kan fortsätta köra även om ett fel uppstår, istället för att krascha.  Innanför `try`-blocket placeras koden som kan orsaka en exception.  `except`-blocket innehåller koden som ska köras om en exception uppstår.  Man kan specificera vilken typ av exception som ska fångas, eller fånga generella exceptions av typen `Exception`.  Nyckelordet `raise` används för att medvetet skapa en exce